## TODO
 - Make a list of important dates (holidays, fast sundays, stake conferences)
 - Add a check for all future-scheduled dates that they don't have any hymns that were sung recently. This would happen if a hymn got changed after the schedule was made.

# Steps to use:
 1. Make sure the history is accurate for the past few weeks
 2. Check if there are any holidays, fast sundays, or special meetings coming up
 3. Run most of the notebook to generate suggestions
 4. Select hymns, enter on Edify
 5. Add selected hymns to history
 6. Add hymns to Gospel Library
 7. Update ward program, if needed, using Gospel Library as reference


In [1]:
import datetime as dt

import numpy as np
import polars as pl

pl.Config().set_tbl_rows(30)
pl.Config().set_fmt_str_lengths(50)

polars.config.Config

In [2]:
def load_history() -> pl.DataFrame:
    history_df = pl.read_csv('history.csv', infer_schema_length=None)
    history_df = history_df.with_columns(
        pl.col("date").str.strptime(pl.Date, "%m/%d/%Y").alias("date"),
    ).drop('name').drop_nulls()
    return history_df

def error_check_history() -> None:
    """Check that no future dates are duplicates with any dates since 8 weeks ago."""
    history_df = load_history()
    num_weeks_cutoff = 8
    today = dt.datetime.today().date()
    lookback_date = today - dt.timedelta(days=7 * num_weeks_cutoff)
    recent_and_future_history = history_df.filter(pl.col("date") >= lookback_date)
    value_counts = recent_and_future_history["id"].value_counts()
    recent_and_future_history = recent_and_future_history.join(
        value_counts, on="id", how="left"
    ).rename({"count": "recent_count"})

    future_history = recent_and_future_history.filter(pl.col("date") >= today)
    duplicates = future_history.filter(pl.col("recent_count") > 1)
    if len(duplicates) > 0:
        display(duplicates)
        print(">>> WARNING! DUPLICATES FOUND! <<<\n")
        raise RuntimeWarning(">>> WARNING! DUPLICATES FOUND! <<<")

def add_last_sung_col(df):
    history_df = load_history()
    last_sung = history_df.group_by('id').agg(pl.max('date').alias('last_sung'))
    df = df.join(last_sung, on='id', how='left')
    return df


def get_ranking_score(df: pl.DataFrame, type_col: str, popularity_weight=0.5, noise=0.0) -> pl.Series:
    score_df = df.filter((pl.col(type_col) == 1) & (pl.col('flagged').fill_null(0) != 1))
    weeks_since_fill_value = 100
    # Add days since last sung column
    score_df = score_df.with_columns(
        ((pl.lit(dt.datetime.today()) - pl.col('last_sung')).dt.total_days()//7)
        .fill_null(weeks_since_fill_value)
        .alias('weeks_since')
    )

    midpoint = 7
    slope = 2

    def popularity_func(x): return np.round(
        1 / (1 + np.exp(-slope * (x - midpoint))), 2)
    weeks_cutoff = 8
    score_df = score_df.with_columns((pl.col('popularity') + pl.col('popularity_adjustment').fill_null(0)).alias('adj_popularity'))
    score_df = score_df.with_columns(
        pl.col('adj_popularity').map_elements(popularity_func).alias('popularity_score'),
        pl.when(pl.col('weeks_since') > weeks_cutoff)
        .then((pl.col('weeks_since') / (weeks_since_fill_value/2)).round(3))
        .otherwise(-100).alias('weeks_since_score'),
        pl.Series(np.round(np.random.uniform(-noise, noise,
                  len(score_df)), 2)).alias('noise'),
    ).with_columns(
        (pl.col('popularity_score') * popularity_weight + pl.col('weeks_since_score')
         * (1 - popularity_weight)).alias('score') + pl.col('noise')
    )

    score_df = score_df.select([
        'id', 'name', 'length', 'adj_popularity', 'weeks_since', 'popularity_score', 'weeks_since_score', 'score'
    ]).sort('score', descending=True)

    return score_df

In [3]:
error_check_history()

date,slot,id,recent_count
date,str,i64,u32
2024-06-02,"""opening""",6,2


>>> WARNING! DUPLICATES FOUND! <<<



RuntimeWarning: 

In [5]:
df = pl.read_csv("hymns.csv")
df = add_last_sung_col(df)

## Sacrament Hymn

In [6]:
sacrament_ranking = get_ranking_score(df, 'is_sacrament', noise=0.1)
display(sacrament_ranking.head(10))

id,name,length,adj_popularity,weeks_since,popularity_score,weeks_since_score,score
i64,str,str,i64,i64,f64,f64,f64
1008,"""Bread of Life, Living Water""","""3:10""",11,100,1.0,2.0,1.44
1007,"""As Bread Is Broken""","""2:06""",13,100,1.0,2.0,1.4
185,"""Reverently and Meekly Now""","""5:06""",8,42,0.88,0.84,0.95
169,"""As Now We Take the Sacrament""","""3:01""",10,30,1.0,0.6,0.9
181,"""Jesus of Nazareth, Savior and King""","""2:52""",10,25,1.0,0.5,0.82
192,"""He Died! The Great Redeemer Died""","""3:05""",10,20,1.0,0.4,0.77
187,"""God Loved Us, So He Sent His Son""","""1:46""",10,19,1.0,0.38,0.75
197,"""O Savior, Thou Who Wearest a Crown""","""4:16""",6,67,0.12,1.34,0.75
183,"""In Rememberance of Thy Suffering""","""3:25""",10,15,1.0,0.3,0.74


## General Hymns

In [11]:
general_ranking = get_ranking_score(df, 'is_general', noise=0.15, popularity_weight=0.7)
display(general_ranking.head(30))

id,name,length,adj_popularity,weeks_since,popularity_score,weeks_since_score,score
i64,str,str,i64,i64,f64,f64,f64
1001,"""Come, Thou Fount of Every Blessing""","""2:32""",13,100,1.0,2.0,1.16
1003,"""It Is Well With My Soul""","""3:21""",12,100,1.0,2.0,1.16
134,"""I Believe in Christ""","""5:00""",8,47,0.88,0.94,0.928
237,"""Do What Is Right""","""3:03""",8,31,0.88,0.62,0.922
98,"""I Need Thee Every Hour""","""4:07""",9,18,0.98,0.36,0.904
66,"""Rejoice, the Lord Is King!""","""2:19""",9,17,0.98,0.34,0.898
277,"""As I Search the Holy Scriptures""","""2:16""",8,48,0.88,0.96,0.894
220,"""Lord, I Would Follow Thee""","""3:01""",10,21,1.0,0.42,0.876
130,"""Be Thou Humble""","""1:56""",7,70,0.5,1.4,0.87


# Add to History

In [ ]:
raise KeyboardInterrupt("This is here to stop you from running the whole notebook.")

KeyboardInterrupt: This is here to stop you from running the whole notebook.

In [ ]:
def name_from_id(id, trim_parens=False):
    name_row = df.filter(pl.col("id") == id).select("name")
    if len(name_row) != 1:
        raise ValueError(f"Expected 1 row, got {len(name_row)}")
    name = name_row["name"].item()

    if trim_parens:
        if name.endswith(")") and "(" in name:
            name = name[:name.rfind("(")].strip()
    
    return name


def format_for_csv(date, opening, sacrament, intermediate, closing):
    str_list = []
    if opening:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},opening,{opening},\"{name_from_id(opening)}\""
        )
    if sacrament:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},sacrament,{sacrament},\"{name_from_id(sacrament)}\""
        )
    if intermediate:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},intermediate,{intermediate},\"{name_from_id(intermediate)}\""
        )
    if closing:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},closing,{closing},\"{name_from_id(closing)}\""
        )
    csv_str = "\n".join(str_list)
    return csv_str


def format_for_paste(opening, sacrament, intermediate, closing):
    str_list = []
    if opening:
        str_list.append(
            f"{opening} - {name_from_id(opening, trim_parens=True)}",
        )
    else:
        str_list.append("~")
    if sacrament:
        str_list.append(
            f"{sacrament} - {name_from_id(sacrament, trim_parens=True)}",
        )
    else:
        str_list.append("~")
    if intermediate:
        str_list.append(
            f"{intermediate} - {name_from_id(intermediate, trim_parens=True)}",
        )
    else:
        str_list.append("~")
    if closing:
        str_list.append(
            f"{closing} - {name_from_id(closing, trim_parens=True)}",
        )
    else:
        str_list.append("~")
    paste_str = "\n".join(str_list)
    return paste_str

date = dt.date(2024, 6, 2) # EDIT ME!
opening = 6
sacrament = None
intermediate = 301
closing = 85

In [ ]:
csv_str = format_for_csv(date, opening, sacrament, intermediate, closing)
print("Does this look right?")
print(csv_str)

Does this look right?
06/02/2024,opening,6,"Redeemer of Israel"
06/02/2024,intermediate,301,"I Am a Child of God"
06/02/2024,closing,85,"How Firm a Foundation"


In [ ]:
# Append to history csv
with open('history.csv', 'a') as f:
    f.write(csv_str)
    f.write("\n")

In [ ]:
paste_str = format_for_paste(opening, sacrament, intermediate, closing)
print(paste_str)

6 - Redeemer of Israel
~
301 - I Am a Child of God
85 - How Firm a Foundation
